In [ ]:
# Missing function definition added for compatibility
def prepare_claude_prompt(column_samples: Dict[str, List[Any]], table_name: str) -> str:
    """Prepare prompt for Claude API with column samples"""
    
    prompt = f"""You are a database schema expert. Analyze the following dataset columns to generate optimized MySQL schema and business descriptions.

Dataset: {table_name}

For each column, I provide sample values. Generate the most appropriate MySQL-compatible SQL data type and a professional business description (max 400 chars).

Column Analysis:
"""
    
    for col_name, samples in column_samples.items():
        prompt += f"""
--- {col_name} ---
Sample Values: {samples}
"""
    
    prompt += """

Please provide your analysis in JSON format:
{
  "columns": {
    "column_name": {
      "sql_type": "MySQL data type with size/precision",
      "nullable": true/false,
      "description": "Business description (max 400 chars)"
    }
  }
}

Guidelines:
- Use appropriate MySQL types: TINYINT, INT, BIGINT, DECIMAL(p,s), VARCHAR(n), DATE, BOOLEAN
- Consider semantic meaning of column names
- Write clear business descriptions
- Use NOT NULL for columns that shouldn't be empty"""
    
    return prompt

# Component One
## Goal
Automatically detect and parse CSV structures with comprehensive error handling, supporting diverse formatting edge cases.


### Possible Challenges

- Missing or ambiguous headers  
- Different delimiter types  
- Malformed rows  
- Issues with heuristics for determining data types  


### Success Criteria

- **No data loss:** All rows and columns must be preserved, even if malformed (e.g., problematic rows are quarantined or logged).  
- **Performance tracking:** Monitor and record processing time.  
- **Traceability:** Maintain logs of errors and parsing issues.  


### Technology

- **Polars:** Chosen for its superior speed and memory efficiency compared to Pandas.

In [15]:
# Import necessary libraries
import polars as pl
import os
import time
from pathlib import Path
from typing import List, Dict
import random


In [16]:

def format_duration(seconds: float) -> str:
    if seconds < 1:
        return f"{seconds*1000:.1f}ms"
    elif seconds < 60:
        return f"{seconds:.2f}s"
    else:
        minutes = int(seconds // 60)
        remaining_seconds = seconds % 60
        return f"{minutes}m {remaining_seconds:.1f}s"

In [17]:
def analyze_file(file_path: Path, sample_lines: int = 5, use_random=False, max_random_lines=5000) -> Dict:
    print(f"🔍 Analyzing: {file_path.name}")
    lines = []

    with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
        all_lines = [line.strip() for _, line in zip(range(max_random_lines), f)]

    # Filter out empty or comment lines
    all_lines = [l for l in all_lines if l and not l.startswith("#")]

    if use_random:
        lines = random.sample(all_lines, min(len(all_lines), sample_lines))
    else:
        lines = all_lines[:sample_lines]

    if not lines:
        return {'error': 'empty_file'}

    first_line = lines[0]
    delimiters = [',', ';', '\t', '|']
    counts = {d: first_line.count(d) for d in delimiters}
    delimiter = max(counts, key=counts.get)
    expected_cols = counts[delimiter] + 1

    return {
        'delimiter': delimiter,
        'expected_columns': expected_cols,
        'sample_lines': lines,
        'likely_header': first_line.split(delimiter)
    }


In [18]:
def load_csv_fast(file_path: Path, delimiter: str) -> pl.DataFrame:
    try:
        df = pl.read_csv(
            file_path, 
            separator=delimiter,
            ignore_errors=True,
            encoding="utf8-lossy",
            infer_schema_length=100
        )
        return df
    except Exception as e:
        print(f"❌ Failed: {e}")
        return None


In [19]:
def fix_merged_columns(df: pl.DataFrame, file_analysis: Dict) -> pl.DataFrame:
    if df.shape[1] != 1:
        return df
    
    col = df.columns[0]
    data = df[col].to_list()
    rows = [row.split(file_analysis['delimiter']) for row in data if row]
    
    headers = file_analysis['likely_header']
    max_cols = max(len(row) for row in rows)
    while len(headers) < max_cols:
        headers.append(f"COL_{len(headers)+1}")
    
    col_data = {h: [r[i] if i < len(r) else None for r in rows] for i, h in enumerate(headers)}
    return pl.DataFrame(col_data)


In [20]:
def clean_string_columns(df: pl.DataFrame) -> pl.DataFrame:
    string_cols = [c for c in df.columns if df[c].dtype == pl.Utf8]
    if not string_cols:
        return df
    
    expressions = [
        pl.col(c).str.strip_chars().replace("", None).alias(c) if c in string_cols else pl.col(c)
        for c in df.columns
    ]
    return df.select(expressions)

In [24]:
def analyze_dataframe(df: pl.DataFrame, name="Dataset"):
    print(f"\n📊 {name.upper()}")
    print(f"  ✔ Rows × Columns: {df.shape}")
    print(f"  ✔ Memory Usage: {df.estimated_size('mb'):.1f} MB")

    if df.shape[1] == 1:
        print("  ⚠️ Warning: Only one column detected — possible delimiter issue.")
    else:
        print(f"  ✔ Column Count OK: {df.shape[1]} columns")

    print(f"  ✔ Column Types & Null %:")
    for i, col in enumerate(df.columns):
        null_pct = (df[col].null_count() / df.shape[0]) * 100
        dtype = str(df[col].dtype)
        print(f"    {i+1:2d}. {col:<20} {dtype:<10} Nulls: {null_pct:5.1f}%")
        if null_pct > 50:
            print(f"       ⚠️ Warning: Over 50% nulls in column '{col}'")

    print("\n📋 Sample (first 3 rows):")
    try:
        display(df.head(3))  # For notebooks
    except:
        print(df.head(3))    # Fallback for script mode

    print(f"✅ {name.upper()} loaded successfully.")


In [25]:
def process_csv_file(file_path: str):
    file_path = Path(file_path)
    analysis = analyze_file(file_path)
    
    if 'error' in analysis:
        print(f"❌ Skipping {file_path.name}: {analysis['error']}")
        return None
    
    df = load_csv_fast(file_path, analysis['delimiter'])
    if df is None:
        return None
    
    # Fix merged column if needed
    if df.shape[1] == 1 and analysis['delimiter'] in df.columns[0]:
        df = fix_merged_columns(df, analysis)
    
    df = clean_string_columns(df)
    analyze_dataframe(df, name=file_path.stem)
    return df

In [26]:
file_paths = [
    "/Users/tomasnagy/Scavenger AI/sap.csv",
    "/Users/tomasnagy/Scavenger AI/insurance.csv"
]

datasets = {}
start = time.time()

for path in file_paths:
    name = Path(path).stem
    print(f"\n📂 Processing: {name}")
    df = process_csv_file(path)
    if df is not None:
        datasets[name] = df

print(f"\n✅ Completed in {format_duration(time.time() - start)}")


📂 Processing: sap
🔍 Analyzing: sap.csv

📊 SAP
  ✔ Rows × Columns: (100000, 64)
  ✔ Memory Usage: 25.5 MB
  ✔ Column Count OK: 64 columns
  ✔ Column Types & Null %:
     1. VBELN                Int64      Nulls:   0.0%
     2. GPAG                 Int64      Nulls:   0.0%
     3. JPARVWGPAG           Int64      Nulls:   0.0%
     4. ANGDT                Int64      Nulls:   0.0%
     5. BNDDT                Int64      Nulls:   0.0%
     6. VBTYP                String     Nulls:   0.0%
     7. TRVOG                Int64      Nulls:   0.0%
     8. AUART                String     Nulls:   0.0%
     9. AUARTGRP             Int64      Nulls:   0.0%
    10. WAERK                String     Nulls:   0.0%
    11. NETWR                Float64    Nulls:   0.0%
    12. VKORG                Int64      Nulls:   0.0%
    13. VTWEG                String     Nulls:   0.0%
    14. SPART                String     Nulls:   0.0%
    15. VKGRP                String     Nulls: 100.0%
       ⚠️ Warning: Over 5

VBELN,GPAG,JPARVWGPAG,ANGDT,BNDDT,VBTYP,TRVOG,AUART,AUARTGRP,WAERK,NETWR,VKORG,VTWEG,SPART,VKGRP,VKBUR,EXEMPLART,GSBER,GSKST,KNUMV,FAKSP,KALSM,KURST,BSTNK,BSARK,BSTDK,BSTZD,IHREZ,STAFO,STWAE,XIMMATRIK,IMMATDAT,XSEPFKKOPF,XREMRECHT,REMSP,KORRGRD,REMVON,REMBIS,REFBELEG,REMDATUM,REMSCHEIN,IVWDATUM,POSNR_LAST,POSEX_LAST,KPOSN_LAST,XFKBASAUFT,XFKBASLIEF,XJKSOFAKT,ERFUSER,ERFDATE,ERFTIME,AENUSER,AENDATE,AENTIME,XWBZABO,XFKVDICHT,XSTATARC,XNOMESS,KALSM_AMO,XRENEWAL,AMORTN,REKLERGB,REKLTYP,REKLDATUM
i64,i64,i64,i64,i64,str,i64,str,i64,str,f64,i64,str,str,str,str,str,str,str,i64,str,str,str,str,str,i64,str,str,str,str,str,i64,str,str,str,str,i64,i64,str,i64,i64,i64,i64,i64,i64,str,str,str,str,i64,i64,str,i64,i64,str,str,str,str,str,str,str,str,i64,i64
31644090,93152002,1,0,0,"""K""",7,"""GABO""",6,"""EUR""",0.0,210,"""AB""","""ZT""",null,null,null,null,null,22448148,null,"""ZSTDA1""","""EURO""",null,null,0,null,null,null,null,null,0,null,null,null,null,0,0,null,0,0,0,0,0,1,"""X""",null,null,"""BURGERTB""",20190715,74745,null,0,0,null,null,null,null,null,null,null,null,1,20190715
31644091,96357515,1,0,0,"""K""",7,"""GABO""",6,"""EUR""",-1.96,210,"""AB""","""ZT""",null,null,null,null,null,22448149,null,"""ZSTDA1""","""EURO""",null,null,0,null,null,null,null,null,0,null,null,null,null,0,0,null,0,0,0,0,0,1,"""X""",null,null,"""SCHMIDSO""",20190715,74745,"""SCHMIDSO""",20190715,74914,null,null,null,null,null,null,null,null,4,20190715
31644092,93107637,1,0,0,"""K""",7,"""GABO""",6,"""EUR""",0.0,210,"""AB""","""ZT""",null,null,null,null,null,22448150,null,"""ZSTDA1""","""EURO""",null,null,0,null,null,null,null,null,0,null,null,null,null,0,0,null,0,0,0,0,0,1,"""X""",null,null,"""KORNHAUSLER""",20190715,74755,null,0,0,null,null,null,null,null,null,null,null,1,20190715


✅ SAP loaded successfully.

📂 Processing: insurance
🔍 Analyzing: insurance.csv

📊 INSURANCE
  ✔ Rows × Columns: (976, 6)
  ✔ Memory Usage: 0.0 MB
  ✔ Column Count OK: 6 columns
  ✔ Column Types & Null %:
     1. age                  Int64      Nulls:   0.0%
     2. sex                  Int64      Nulls:   0.0%
     3. bmi                  Float64    Nulls:   0.0%
     4. children             Int64      Nulls:   0.0%
     5. smoker               Int64      Nulls:   0.0%
     6. charges              Float64    Nulls:   0.0%

📋 Sample (first 3 rows):


age,sex,bmi,children,smoker,charges
i64,i64,f64,i64,i64,f64
19,1,27.9,0,1,16884.924
18,0,33.77,1,0,1725.5523
28,0,33.0,3,0,4449.462


✅ INSURANCE loaded successfully.

✅ Completed in 136.5ms


# Component Two

## Goal
Analyze the data,  compute basic descriptive statistics with a focus on missing value treatment.

### Possible Challenges

    - Missing Values: analyse Develop a strategy for handling missing data.
    - Decide how to select an imputation method (statistical vs. ML-based).
    - Identify which columns require correction.
    - Establish criteria to evaluate the effectiveness of the chosen strategy.

- **Datatypes:**  
    - Build heuristics for accurate data type identification.

### Success Criteria

- **Minimize Missing Data:**  
    The proportion of missing data should be reduced as much as possible through appropriate handling and imputation.

### Technology

- **Polars:**  
    Utilized for efficient data analysis and manipulation.


In [38]:
def analyze_dataset_structure_and_nulls(df: pl.DataFrame, name: str):
    print(f"\n📊 {name.upper()}")
    print("─" * 60)
    
    # Shape and Memory
    print(f"✔️  Rows × Columns:       {df.shape[0]} × {df.shape[1]}")
    print(f"✔️  Memory Usage:         {df.estimated_size('mb'):.1f} MB")

    # Check for delimiter issue
    if df.shape[1] == 1:
        print("⚠️  Warning: Only one column detected — possible delimiter issue (e.g., expected ';' but read as ',').")
    else:
        print(f"✔️  Column Count OK:      {df.shape[1]} columns")

    # Column overview with enhanced statistics
    print("\n📑 Column Analysis:")
    numeric_cols = []
    string_cols = []
    
    for i, col in enumerate(df.columns):
        try:
            null_count = df[col].null_count()
            null_pct = (null_count / df.shape[0]) * 100
            dtype = str(df[col].dtype)
            
            print(f"  {i+1:>2}. {col:<25} {dtype:<12}  Nulls: {null_pct:5.1f}% ({null_count:,})")
            
            # Collect column types for detailed stats
            if df[col].dtype in [pl.Int64, pl.Int32, pl.Float64, pl.Float32]:
                numeric_cols.append(col)
            elif df[col].dtype == pl.Utf8:
                string_cols.append(col)
                
            if null_pct > 50:
                print(f"      ⚠️ Over 50% missing values in '{col}'")
                
        except Exception as e:
            print(f"      ❌ Error analyzing column '{col}': {e}")

    # Descriptive Statistics for Numeric Columns
    if numeric_cols:
        print(f"\n📈 NUMERIC COLUMNS STATISTICS ({len(numeric_cols)} columns)")
        print("─" * 80)
        
        for col in numeric_cols:
            try:
                stats = df[col].describe()
                non_null_count = df.shape[0] - df[col].null_count()
                
                print(f"\n🔢 {col.upper()}")
                print(f"   Count (non-null): {non_null_count:,}")
                print(f"   Mean:            {df[col].mean():.4f}")
                print(f"   Std Dev:         {df[col].std():.4f}")
                print(f"   Min:             {df[col].min()}")
                print(f"   25th Percentile: {df[col].quantile(0.25)}")
                print(f"   Median (50th):   {df[col].median()}")
                print(f"   75th Percentile: {df[col].quantile(0.75)}")
                print(f"   Max:             {df[col].max()}")
                
                # Check for outliers using IQR method
                q1 = df[col].quantile(0.25)
                q3 = df[col].quantile(0.75)
                iqr = q3 - q1
                lower_bound = q1 - 1.5 * iqr
                upper_bound = q3 + 1.5 * iqr
                outliers = df.filter((pl.col(col) < lower_bound) | (pl.col(col) > upper_bound)).shape[0]
                
                if outliers > 0:
                    print(f"   ⚠️ Potential outliers: {outliers:,} values outside [{lower_bound:.2f}, {upper_bound:.2f}]")
                    
            except Exception as e:
                print(f"   ❌ Error calculating stats for '{col}': {e}")

    # String Column Analysis
    if string_cols:
        print(f"\n📝 STRING COLUMNS ANALYSIS ({len(string_cols)} columns)")
        print("─" * 80)
        
        for col in string_cols:
            try:
                non_null_count = df.shape[0] - df[col].null_count()
                unique_count = df[col].n_unique()
                
                print(f"\n📄 {col.upper()}")
                print(f"   Count (non-null):    {non_null_count:,}")
                print(f"   Unique values:       {unique_count:,}")
                print(f"   Uniqueness ratio:    {(unique_count/non_null_count)*100:.1f}%")
                
                # Show most frequent values
                if unique_count > 0:
                    top_values = (df[col]
                                .value_counts()
                                .head(5)
                                .sort('count', descending=True))
                    
                    print("   Top 5 values:")
                    for row in top_values.iter_rows():
                        value, count = row
                        pct = (count / non_null_count) * 100
                        print(f"     '{value}': {count:,} ({pct:.1f}%)")
                        
            except Exception as e:
                print(f"   ❌ Error analyzing string column '{col}': {e}")

    # Overall Data Quality Summary
    total_cells = df.shape[0] * df.shape[1]
    total_nulls = sum(df[col].null_count() for col in df.columns)
    completeness = ((total_cells - total_nulls) / total_cells) * 100
    
    print(f"\n📋 DATA QUALITY SUMMARY")
    print("─" * 40)
    print(f"✔️  Total cells:          {total_cells:,}")
    print(f"✔️  Non-null cells:       {total_cells - total_nulls:,}")
    print(f"✔️  Data completeness:    {completeness:.1f}%")
    print(f"✔️  Numeric columns:      {len(numeric_cols)}")
    print(f"✔️  String columns:       {len(string_cols)}")

    # Sample rows
    print("\n📋 Sample (first 3 rows):")
    print("─" * 60)
    try:
        sample = df.head(3)
        header = " | ".join(sample.columns)
        types = " | ".join(str(df[col].dtype) for col in sample.columns)
        print(f"| {header} |")
        print(f"| {types} |")
        for row in sample.iter_rows():
            print("| " + " | ".join(str(x) if x is not None else "null" for x in row) + " |")
    except Exception as e:
        print(f"⚠️ Could not print sample: {e}")

    print(f"\n✅ {name.upper()} analysis completed.")
    print("─" * 60)

In [40]:
def process_csv_file_simple(file_path: str):
    """Simplified version without analysis output"""
    file_path = Path(file_path)
    analysis = analyze_file(file_path)
    
    if 'error' in analysis:
        print(f"❌ Skipping {file_path.name}: {analysis['error']}")
        return None
    
    df = load_csv_fast(file_path, analysis['delimiter'])
    if df is None:
        return None
    
    # Fix merged column if needed
    if df.shape[1] == 1 and analysis['delimiter'] in df.columns[0]:
        df = fix_merged_columns(df, analysis)
    
    df = clean_string_columns(df)
    return df

# === Final Execution: Component One and Component Two ===
file_paths = [
    "/Users/tomasnagy/Scavenger AI/insurance.csv",
    "/Users/tomasnagy/Scavenger AI/sap.csv"

]

datasets = {}
start = time.time()

for path in file_paths:
    name = Path(path).stem
    print(f"\n📂 Processing: {name.upper()}")
    
    # Component One: CSV Structure Detection (without analysis)
    df = process_csv_file_simple(path)
    
    if df is not None:
        datasets[name] = df
        
        # Component Two: Data Quality Analysis
        analyze_dataset_structure_and_nulls(df, name)

print(f"\n✅ All components completed in {format_duration(time.time() - start)}")


📂 Processing: INSURANCE
🔍 Analyzing: insurance.csv

📊 INSURANCE
────────────────────────────────────────────────────────────
✔️  Rows × Columns:       976 × 6
✔️  Memory Usage:         0.0 MB
✔️  Column Count OK:      6 columns

📑 Column Analysis:
   1. age                       Int64         Nulls:   0.0% (0)
   2. sex                       Int64         Nulls:   0.0% (0)
   3. bmi                       Float64       Nulls:   0.0% (0)
   4. children                  Int64         Nulls:   0.0% (0)
   5. smoker                    Int64         Nulls:   0.0% (0)
   6. charges                   Float64       Nulls:   0.0% (0)

📈 NUMERIC COLUMNS STATISTICS (6 columns)
────────────────────────────────────────────────────────────────────────────────

🔢 AGE
   Count (non-null): 976
   Mean:            39.6670
   Std Dev:         14.2353
   Min:             18
   25th Percentile: 27.0
   Median (50th):   40.0
   75th Percentile: 52.0
   Max:             64

🔢 SEX
   Count (non-null): 976
   

In [54]:
import json
import requests
from typing import Dict, List, Tuple, Any
import time

# Claude API configuration
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Claude API configuration
CLAUDE_API_KEY = os.getenv('CLAUDE_API_KEY')
if not CLAUDE_API_KEY:
    raise ValueError(
        "🔑 CLAUDE_API_KEY environment variable is not set!\n"
        "Please set it using: export CLAUDE_API_KEY='your-api-key-here'\n"
        "Or create a .env file with CLAUDE_API_KEY=your-api-key-here"
    )
CLAUDE_API_URL = "https://api.anthropic.com/v1/messages"

def analyze_column_metadata(df: pl.DataFrame) -> Dict[str, Dict]:
    """Analyze comprehensive metadata for each column using Polars"""
    metadata = {}
    
    for col in df.columns:
        col_data = df[col]
        non_null_data = col_data.drop_nulls()
        
        # Basic statistics
        total_count = df.shape[0]
        null_count = col_data.null_count()
        non_null_count = total_count - null_count
        null_percentage = (null_count / total_count) * 100 if total_count > 0 else 0
        unique_count = col_data.n_unique()
        
        # Initialize column metadata
        col_meta = {
            'column_name': col,
            'total_count': total_count,
            'null_count': null_count,
            'non_null_count': non_null_count,
            'null_percentage': round(null_percentage, 2),
            'unique_count': unique_count,
            'uniqueness_ratio': round((unique_count / non_null_count) * 100, 2) if non_null_count > 0 else 0,
            'polars_dtype': str(col_data.dtype),
            'sample_values': [],
            'most_common_value': None,
            'most_common_count': 0,
            'candidate_types': [],
            'min_value': None,
            'max_value': None,
            'avg_length': None
        }
        
        if non_null_count > 0:
            # Sample values (up to 5)
            sample_size = min(5, non_null_count)
            if sample_size == non_null_count:
                col_meta['sample_values'] = non_null_data.to_list()
            else:
                col_meta['sample_values'] = non_null_data.sample(sample_size).to_list()
            
            # Most common value
            try:
                value_counts = col_data.value_counts().head(1)
                if value_counts.shape[0] > 0:
                    row = value_counts.row(0)
                    col_meta['most_common_value'] = row[0]
                    col_meta['most_common_count'] = row[1]
            except:
                pass
            
            # Type-specific analysis
            if col_data.dtype in [pl.Int64, pl.Int32, pl.Int16, pl.Int8]:
                col_meta['min_value'] = col_data.min()
                col_meta['max_value'] = col_data.max()
                col_meta['candidate_types'] = ['INT', 'BIGINT']
                
            elif col_data.dtype in [pl.Float64, pl.Float32]:
                col_meta['min_value'] = col_data.min()
                col_meta['max_value'] = col_data.max()
                col_meta['candidate_types'] = ['DECIMAL', 'FLOAT', 'DOUBLE']
                
            elif col_data.dtype == pl.Utf8:
                # String analysis
                lengths = non_null_data.str.len_chars()
                col_meta['min_value'] = lengths.min()
                col_meta['max_value'] = lengths.max()
                col_meta['avg_length'] = round(lengths.mean(), 1) if lengths.len() > 0 else 0
                
                # Detect candidate types for strings
                candidates = ['VARCHAR']
                
                # Check if it could be a date (common patterns)
                sample_str_values = [str(v) for v in col_meta['sample_values'][:3]]
                if any(len(str(v)) == 8 and str(v).isdigit() for v in sample_str_values):
                    candidates.append('DATE')  # YYYYMMDD format
                elif any('/' in str(v) or '-' in str(v) for v in sample_str_values):
                    candidates.append('DATE')  # Common date separators
                
                # Check if it could be boolean
                unique_vals = set(str(v).upper() for v in col_meta['sample_values'])
                if unique_vals.issubset({'0', '1', 'TRUE', 'FALSE', 'T', 'F', 'YES', 'NO', 'Y', 'N'}):
                    candidates.append('BOOLEAN')
                
                # Check if it could be numeric (stored as string)
                if all(str(v).replace('.', '').replace('-', '').isdigit() for v in sample_str_values if v):
                    candidates.extend(['INT', 'DECIMAL'])
                
                col_meta['candidate_types'] = candidates
                
            elif col_data.dtype == pl.Boolean:
                col_meta['candidate_types'] = ['BOOLEAN', 'TINYINT']
                
            else:
                col_meta['candidate_types'] = ['VARCHAR']
        
        else:
            # All null column
            col_meta['sample_values'] = ['NULL']
            col_meta['candidate_types'] = ['VARCHAR']
        
        metadata[col] = col_meta
    
    return metadata

def extract_column_samples(df: pl.DataFrame, sample_size: int = 5) -> Dict[str, List[Any]]:
    """Extract sample values from each column for LLM analysis"""
    samples = {}
    
    for col in df.columns:
        # Get non-null values first
        non_null_values = df[col].drop_nulls()
        
        if non_null_values.len() > 0:  # Changed from .height to .len()
            # Take sample_size random samples (or all if less than sample_size)
            sample_count = min(sample_size, non_null_values.len())  # Changed from .height to .len()
            if sample_count == non_null_values.len():  # Changed from .height to .len()
                samples[col] = non_null_values.to_list()
            else:
                samples[col] = non_null_values.sample(sample_count).to_list()
        else:
            samples[col] = ["NULL"]  # Indicate all nulls
    
    return samples

def prepare_enhanced_claude_prompt(column_metadata: Dict[str, Dict], table_name: str) -> str:
    """Prepare an enhanced prompt with rich metadata for Claude to analyze column types and generate descriptions"""
    
    prompt = f"""You are a database schema expert. Analyze the following dataset columns with their comprehensive metadata to generate optimized SQL schema and business descriptions.

Dataset: {table_name}

For each column below, I provide:
- Column name and semantic context
- Sample values
- Statistical metadata (nulls, uniqueness, min/max, most common value)
- Candidate data types detected by analysis
- Current Polars data type

Your task: Generate the most appropriate MySQL-compatible SQL data type and a professional business description.

Column Analysis:
"""
    
    for col_name, meta in column_metadata.items():
        prompt += f"""
--- {col_name} ---
- Sample Values: {meta['sample_values']}
- Most Common: {meta['most_common_value']} (appears {meta['most_common_count']} times)
- Nulls: {meta['null_percentage']}% ({meta['null_count']}/{meta['total_count']})
- Unique Values: {meta['unique_count']} ({meta['uniqueness_ratio']}% unique)
- Candidate Types: {meta['candidate_types']}
- Current Type: {meta['polars_dtype']}"""
        
        if meta['min_value'] is not None and meta['max_value'] is not None:
            if meta['polars_dtype'] == 'Utf8':
                prompt += f"\n- String Length: min={meta['min_value']}, max={meta['max_value']}, avg={meta['avg_length']}"
            else:
                prompt += f"\n- Value Range: {meta['min_value']} to {meta['max_value']}"
    
    prompt += """

Please provide your analysis in the following JSON format:
{
  "columns": {
    "column_name": {
      "sql_type": "optimized MySQL data type with proper size/precision",
      "nullable": true/false,
      "description": "Professional business description based on semantic analysis"
    }
  }
}

Guidelines for optimal SQL types:
- Use TINYINT for small integers (0-255)
- Use INT for standard integers 
- Use BIGINT for large integers
- Use DECIMAL(precision,scale) for exact decimals (e.g., DECIMAL(10,2) for currency)
- Use FLOAT/DOUBLE only for scientific data
- Use VARCHAR(optimal_length) based on max string length + buffer
- Use DATE for date values (YYYYMMDD format should be DATE, not INT)
- Use BOOLEAN/TINYINT(1) for binary flags
- Consider NOT NULL for columns with 0% nulls and clear business requirements
- Use semantic understanding: 'user_id' suggests primary key, 'email' suggests unique varchar, 'created_at' suggests timestamp

For descriptions:
- Analyze column name semantically (e.g., 'VBELN' might be 'Sales Document Number')
- Use sample values to understand the business context
- Write professional, clear descriptions explaining business purpose
- Consider data patterns (e.g., all values start with same prefix = identifier/code)"""
    
    return prompt

def call_claude_api_robust(prompt: str, max_retries: int = 3, model: str = "claude-3-sonnet-20240229") -> Dict:
    """Enhanced Claude API call with retries, timeouts, and exponential backoff"""
    
    headers = {
        "Content-Type": "application/json",
        "x-api-key": CLAUDE_API_KEY,
        "anthropic-version": "2023-06-01"
    }
    
    data = {
        "model": model,
        "max_tokens": 4000,  # Increased for larger schemas
        "temperature": 0.1,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ]
    }
    
    for attempt in range(max_retries):
        try:
            print(f"📡 Attempt {attempt + 1}/{max_retries}: Calling Claude API...")
            
            # Make request with timeout
            response = requests.post(
                CLAUDE_API_URL, 
                headers=headers, 
                json=data, 
                timeout=120  # 2 minute timeout
            )
            response.raise_for_status()
            
            result = response.json()
            content = result["content"][0]["text"]
            
            # Handle markdown code blocks - remove ```json and ``` if present
            if "```json" in content:
                content = content.split("```json")[1]
                if "```" in content:
                    content = content.split("```")[0]
            elif "```" in content and content.count("```") >= 2:
                # Handle generic code blocks
                parts = content.split("```")
                if len(parts) >= 3:
                    content = parts[1]
            
            # Extract JSON from the response
            start_idx = content.find('{')
            end_idx = content.rfind('}') + 1
            
            if start_idx != -1 and end_idx > start_idx:
                json_str = content[start_idx:end_idx].strip()
                parsed_json = json.loads(json_str)
                print(f"✅ Successfully parsed response on attempt {attempt + 1}")
                return parsed_json
            else:
                print(f"⚠️ Attempt {attempt + 1}: Could not find JSON in Claude response")
                if attempt == max_retries - 1:
                    print(f"Raw content preview: {content[:200]}...")
                    return None
                    
        except requests.exceptions.Timeout:
            print(f"⏰ Attempt {attempt + 1}: Request timeout (120s exceeded)")
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt  # Exponential backoff: 1s, 2s, 4s
                print(f"⏳ Waiting {wait_time}s before retry...")
                time.sleep(wait_time)
                
        except requests.exceptions.ConnectionError:
            print(f"🌐 Attempt {attempt + 1}: Connection error")
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt
                print(f"⏳ Waiting {wait_time}s before retry...")
                time.sleep(wait_time)
                
        except requests.exceptions.HTTPError as e:
            print(f"🚫 Attempt {attempt + 1}: HTTP error {e.response.status_code}")
            if e.response.status_code == 429:  # Rate limit
                if attempt < max_retries - 1:
                    wait_time = 10 + (2 ** attempt)  # Longer wait for rate limits
                    print(f"🚦 Rate limited. Waiting {wait_time}s before retry...")
                    time.sleep(wait_time)
            elif e.response.status_code >= 500:  # Server error
                if attempt < max_retries - 1:
                    wait_time = 5 + (2 ** attempt)
                    print(f"🔧 Server error. Waiting {wait_time}s before retry...")
                    time.sleep(wait_time)
            else:
                print(f"❌ Non-retryable HTTP error: {e}")
                return None
                
        except json.JSONDecodeError as e:
            print(f"📄 Attempt {attempt + 1}: JSON parse error - {e}")
            if attempt == max_retries - 1:
                print(f"Final JSON content preview: {content[:300] if 'content' in locals() else 'No content'}...")
                return None
                
        except Exception as e:
            print(f"❌ Attempt {attempt + 1}: Unexpected error - {e}")
            if attempt == max_retries - 1:
                import traceback
                print(f"Full traceback: {traceback.format_exc()}")
                return None
    
    print(f"💥 All {max_retries} attempts failed")
    return None

# Legacy function for backward compatibility
def call_claude_api(prompt: str, model: str = "claude-3-sonnet-20240229") -> Dict:
    """Legacy function - now uses robust version"""
    return call_claude_api_robust(prompt, max_retries=3, model=model)

def generate_ddl_from_schema(schema_data: Dict, table_name: str) -> str:
    """Generate CREATE TABLE DDL from Claude schema analysis"""
    
    if not schema_data or 'columns' not in schema_data:
        return "-- Error: Invalid schema data"
    
    ddl = f"CREATE TABLE {table_name} (\n"
    
    column_definitions = []
    for col_name, col_info in schema_data['columns'].items():
        sql_type = col_info.get('sql_type', 'VARCHAR(255)')
        nullable = col_info.get('nullable', True)
        
        null_constraint = " NOT NULL" if not nullable else ""
        column_def = f"  `{col_name}` {sql_type}{null_constraint}"
        column_definitions.append(column_def)
    
    ddl += ",\n".join(column_definitions)
    ddl += "\n);"
    
    return ddl

def generate_column_descriptions(schema_data: Dict) -> str:
    """Generate JSON formatted column descriptions"""
    
    if not schema_data or 'columns' not in schema_data:
        return "{}"
    
    descriptions = {}
    for col_name, col_info in schema_data['columns'].items():
        descriptions[col_name] = col_info.get('description', f"Description for {col_name}")
    
    return json.dumps(descriptions, indent=2)

def analyze_schema_with_claude(df: pl.DataFrame, table_name: str, sample_size: int = 5) -> Tuple[str, str]:
    """Complete pipeline: extract samples, analyze with Claude, generate DDL and descriptions"""
    
    print(f"🔍 Analyzing schema for {table_name} using Claude...")
    
    # Step 1: Extract column samples
    column_samples = extract_column_samples(df, sample_size)
    print(f"✔️ Extracted samples from {len(column_samples)} columns")
    
    # Step 2: Prepare Claude prompt
    prompt = prepare_claude_prompt(column_samples, table_name)
    
    # Step 3: Call Claude for analysis with robust error handling
    print("🤖 Calling Claude API for schema analysis...")
    schema_data = call_claude_api_robust(prompt)
    
    if not schema_data:
        return "-- Error: Failed to generate schema after multiple attempts", "{}"
    
    # Step 4: Generate DDL and descriptions
    ddl = generate_ddl_from_schema(schema_data, table_name)
    descriptions = generate_column_descriptions(schema_data)
    
    print(f"✅ Schema analysis completed for {table_name}")
    
    return ddl, descriptions

def display_schema_results(ddl: str, descriptions: str, table_name: str):
    """Display the generated schema and descriptions in a formatted way"""
    
    print(f"\n📋 CLAUDE-GENERATED SCHEMA FOR {table_name.upper()}")
    print("=" * 70)
    
    print("\n🗂️ DDL (Data Definition Language):")
    print("─" * 50)
    print(ddl)
    
    print("\n📖 Column Descriptions:")
    print("─" * 50)
    print(descriptions)
    
    print("\n" + "=" * 70)

def enhanced_analysis_with_claude_schema(df: pl.DataFrame, name: str):
    """Enhanced analysis that includes Claude-based schema generation"""
    
    # First run your existing analysis
    analyze_dataset_structure_and_nulls(df, name)
    
    # Then add Claude schema analysis
    ddl, descriptions = analyze_schema_with_claude(df, name)
    display_schema_results(ddl, descriptions, name)
    
    return ddl, descriptions

def test_claude_connection() -> bool:
    """Test if Claude API is accessible with robust error handling"""
    test_prompt = "Please respond with just the word 'SUCCESS' if you can read this."
    
    print("🔗 Testing Claude API connection...")
    
    headers = {
        "Content-Type": "application/json",
        "x-api-key": CLAUDE_API_KEY,
        "anthropic-version": "2023-06-01"
    }
    
    data = {
        "model": "claude-3-sonnet-20240229",
        "max_tokens": 10,
        "messages": [{"role": "user", "content": test_prompt}]
    }
    
    try:
        response = requests.post(CLAUDE_API_URL, headers=headers, json=data, timeout=30)
        response.raise_for_status()
        result = response.json()
        content = result["content"][0]["text"]
        
        if "SUCCESS" in content:
            print("✅ Claude API connection successful")
            return True
        else:
            print(f"⚠️ Claude API responded but with unexpected content: {content}")
            return False
            
    except requests.exceptions.Timeout:
        print("❌ Claude API connection failed: Timeout")
        return False
    except requests.exceptions.ConnectionError:
        print("❌ Claude API connection failed: Connection error")
        return False
    except requests.exceptions.HTTPError as e:
        print(f"❌ Claude API connection failed: HTTP {e.response.status_code}")
        return False
    except Exception as e:
        print(f"❌ Claude API connection failed: {e}")
        return False

In [45]:
def extract_column_samples(df: pl.DataFrame, sample_size: int = 5) -> Dict[str, List[Any]]:
    """Extract sample values from each column for LLM analysis"""
    samples = {}
    
    for col in df.columns:
        # Get non-null values first
        non_null_values = df[col].drop_nulls()
        
        if non_null_values.len() > 0:  # Changed from .height to .len()
            # Take sample_size random samples (or all if less than sample_size)
            sample_count = min(sample_size, non_null_values.len())  # Changed from .height to .len()
            if sample_count == non_null_values.len():  # Changed from .height to .len()
                samples[col] = non_null_values.to_list()
            else:
                samples[col] = non_null_values.sample(sample_count).to_list()
        else:
            samples[col] = ["NULL"]  # Indicate all nulls
    
    return samples

# Test the schema generation functionality
print("🧪 Testing Schema Generation...")

# Test Claude connection first
test_claude_connection()

# Test with one of the loaded datasets
if 'datasets' in globals() and datasets:
    dataset_name = list(datasets.keys())[0]
    test_df = datasets[dataset_name]
    
    print(f"\n🔍 Testing schema generation with '{dataset_name}' dataset...")
    ddl, descriptions = analyze_schema_with_claude(test_df, dataset_name, sample_size=3)
    
    display_schema_results(ddl, descriptions, dataset_name)
else:
    print("⚠️ No datasets loaded. Please run the CSV processing cells first.")

🧪 Testing Schema Generation...
✅ Claude API connection successful

🔍 Testing schema generation with 'insurance' dataset...
🔍 Analyzing schema for insurance using Claude...
✔️ Extracted samples from 6 columns
🤖 Calling Claude API for schema analysis...
✅ Claude API connection successful

🔍 Testing schema generation with 'insurance' dataset...
🔍 Analyzing schema for insurance using Claude...
✔️ Extracted samples from 6 columns
🤖 Calling Claude API for schema analysis...
✅ Schema analysis completed for insurance

📋 CLAUDE-GENERATED SCHEMA FOR INSURANCE

🗂️ DDL (Data Definition Language):
──────────────────────────────────────────────────
CREATE TABLE insurance (
  `age` INT NOT NULL,
  `sex` BOOLEAN NOT NULL,
  `bmi` DECIMAL(5,2) NOT NULL,
  `children` INT NOT NULL,
  `smoker` BOOLEAN NOT NULL,
  `charges` DECIMAL(10,4) NOT NULL
);

📖 Column Descriptions:
──────────────────────────────────────────────────
{
  "age": "The age of the insured person in years.",
  "sex": "The biological sex o

In [46]:
# Test the corrected schema generation functionality
print("🧪 Testing Schema Generation...")

# Test Claude connection first
connection_ok = test_claude_connection()

if connection_ok:
    # Test with the insurance dataset (smaller dataset for testing)
    if 'insurance' in datasets:
        print(f"\n🔍 Testing schema generation with 'insurance' dataset...")
        ddl, descriptions = analyze_schema_with_claude(datasets['insurance'], 'insurance', sample_size=3)
        display_schema_results(ddl, descriptions, 'insurance')
    else:
        print("⚠️ Insurance dataset not found in loaded datasets.")
else:
    print("❌ Cannot test schema generation without Claude API connection.")

🧪 Testing Schema Generation...
✅ Claude API connection successful

🔍 Testing schema generation with 'insurance' dataset...
🔍 Analyzing schema for insurance using Claude...
✔️ Extracted samples from 6 columns
🤖 Calling Claude API for schema analysis...
✅ Claude API connection successful

🔍 Testing schema generation with 'insurance' dataset...
🔍 Analyzing schema for insurance using Claude...
✔️ Extracted samples from 6 columns
🤖 Calling Claude API for schema analysis...
✅ Schema analysis completed for insurance

📋 CLAUDE-GENERATED SCHEMA FOR INSURANCE

🗂️ DDL (Data Definition Language):
──────────────────────────────────────────────────
CREATE TABLE insurance (
  `age` INT NOT NULL,
  `sex` BOOLEAN NOT NULL,
  `bmi` DECIMAL(5,2) NOT NULL,
  `children` INT NOT NULL,
  `smoker` BOOLEAN NOT NULL,
  `charges` DECIMAL(10,5) NOT NULL
);

📖 Column Descriptions:
──────────────────────────────────────────────────
{
  "age": "The age of the insured person in years.",
  "sex": "The biological sex o

In [56]:
# Simple test of the corrected extract_column_samples function
print("Testing extract_column_samples function...")

if 'insurance' in datasets:
    # Test the function that was causing the error
    samples = extract_column_samples(datasets['insurance'], sample_size=3)
    print(f"✅ Function works! Extracted {len(samples)} column samples:")
    for col, sample_values in samples.items():
        print(f"  {col}: {sample_values}")
else:
    print("❌ Insurance dataset not found")

Testing extract_column_samples function...
✅ Function works! Extracted 6 column samples:
  age: [57, 41, 56]
  sex: [0, 0, 0]
  bmi: [38.06, 25.555, 26.73]
  children: [2, 2, 0]
  smoker: [0, 1, 0]
  charges: [6082.405, 3206.49135, 5138.2567]


In [57]:
# Test schema generation with SAP dataset
print("🧪 Testing Schema Generation with SAP Dataset...")
print("=" * 60)

# Test Claude connection first
connection_ok = test_claude_connection()

if connection_ok:
    # Test with the SAP dataset
    if 'sap' in datasets:
        print(f"\n🔍 Testing schema generation with 'sap' dataset...")
        
        # First show basic info about the dataset
        sap_df = datasets['sap']
        print(f"📊 SAP Dataset Info:")
        print(f"   Rows: {sap_df.shape[0]:,}")
        print(f"   Columns: {sap_df.shape[1]}")
        print(f"   Column names: {list(sap_df.columns)}")
        
        # Test the extract_column_samples function
        print("\n🔬 Testing extract_column_samples function...")
        try:
            samples = extract_column_samples(sap_df, sample_size=3)
            print(f"✅ Successfully extracted samples from {len(samples)} columns:")
            for col, sample_values in samples.items():
                print(f"   {col}: {sample_values}")
        except Exception as e:
            print(f"❌ Error in extract_column_samples: {e}")
            print("Cannot proceed with schema generation.")
        else:
            # If sample extraction worked, proceed with full schema generation
            print("\n🤖 Proceeding with full schema generation...")
            try:
                ddl, descriptions = analyze_schema_with_claude(sap_df, 'sap', sample_size=3)
                display_schema_results(ddl, descriptions, 'sap')
            except Exception as e:
                print(f"❌ Error in schema generation: {e}")
    else:
        print("⚠️ SAP dataset not found in loaded datasets.")
        print(f"Available datasets: {list(datasets.keys()) if 'datasets' in globals() else 'None'}")
else:
    print("❌ Cannot test schema generation without Claude API connection.")

🧪 Testing Schema Generation with SAP Dataset...
✅ Claude API connection successful

🔍 Testing schema generation with 'sap' dataset...
📊 SAP Dataset Info:
   Rows: 100,000
   Columns: 64
   Column names: ['VBELN', 'GPAG', 'JPARVWGPAG', 'ANGDT', 'BNDDT', 'VBTYP', 'TRVOG', 'AUART', 'AUARTGRP', 'WAERK', 'NETWR', 'VKORG', 'VTWEG', 'SPART', 'VKGRP', 'VKBUR', 'EXEMPLART', 'GSBER', 'GSKST', 'KNUMV', 'FAKSP', 'KALSM', 'KURST', 'BSTNK', 'BSARK', 'BSTDK', 'BSTZD', 'IHREZ', 'STAFO', 'STWAE', 'XIMMATRIK', 'IMMATDAT', 'XSEPFKKOPF', 'XREMRECHT', 'REMSP', 'KORRGRD', 'REMVON', 'REMBIS', 'REFBELEG', 'REMDATUM', 'REMSCHEIN', 'IVWDATUM', 'POSNR_LAST', 'POSEX_LAST', 'KPOSN_LAST', 'XFKBASAUFT', 'XFKBASLIEF', 'XJKSOFAKT', 'ERFUSER', 'ERFDATE', 'ERFTIME', 'AENUSER', 'AENDATE', 'AENTIME', 'XWBZABO', 'XFKVDICHT', 'XSTATARC', 'XNOMESS', 'KALSM_AMO', 'XRENEWAL', 'AMORTN', 'REKLERGB', 'REKLTYP', 'REKLDATUM']

🔬 Testing extract_column_samples function...
✅ Successfully extracted samples from 64 columns:
   VBELN: [

In [52]:
# Comprehensive SAP Dataset Schema Generation Test
print("🔍 COMPREHENSIVE SAP DATASET SCHEMA GENERATION TEST")
print("=" * 70)

# Step 1: Check if datasets are loaded
if 'datasets' not in globals() or not datasets:
    print("⚠️ No datasets found. Loading datasets first...")
    
    # Load datasets using the existing function
    file_paths = [
        "/Users/tomasnagy/Scavenger AI/insurance.csv",
        "/Users/tomasnagy/Scavenger AI/sap.csv"
    ]
    
    datasets = {}
    start = time.time()
    
    for path in file_paths:
        name = Path(path).stem
        print(f"\n📂 Loading: {name.upper()}")
        
        # Use the simple processing function
        df = process_csv_file_simple(path)
        
        if df is not None:
            datasets[name] = df
            print(f"✅ {name} loaded successfully: {df.shape[0]} rows × {df.shape[1]} columns")
        else:
            print(f"❌ Failed to load {name}")
    
    print(f"\n✅ Dataset loading completed in {format_duration(time.time() - start)}")

# Step 2: Test Claude API connection
print("\n🔗 Testing Claude API connection...")
connection_ok = test_claude_connection()

# Step 3: Test SAP dataset specifically
if connection_ok and 'sap' in datasets:
    print("\n🎯 TESTING SAP DATASET SCHEMA GENERATION")
    print("─" * 50)
    
    sap_df = datasets['sap']
    
    # Show dataset info
    print(f"📊 SAP Dataset Overview:")
    print(f"   📈 Rows: {sap_df.shape[0]:,}")
    print(f"   📋 Columns: {sap_df.shape[1]}")
    print(f"   💾 Memory: {sap_df.estimated_size('mb'):.1f} MB")
    print(f"   🏷️ Column names: {list(sap_df.columns)}")
    
    # Test column sampling first
    print("\n🔬 Step 1: Testing column sampling...")
    try:
        samples = extract_column_samples(sap_df, sample_size=5)
        print(f"✅ Successfully extracted samples from {len(samples)} columns:")
        for col, sample_values in samples.items():
            print(f"   📝 {col}: {sample_values}")
    except Exception as e:
        print(f"❌ Error in column sampling: {e}")
        print("Cannot proceed with schema generation.")
    else:
        # Proceed with full schema generation
        print("\n🤖 Step 2: Generating schema with Claude API...")
        try:
            ddl, descriptions = analyze_schema_with_claude(sap_df, 'sap_data', sample_size=5)
            
            # Display results with enhanced formatting
            print("\n" + "=" * 80)
            print("🎉 SAP DATASET SCHEMA GENERATION COMPLETED SUCCESSFULLY!")
            print("=" * 80)
            
            display_schema_results(ddl, descriptions, 'sap_data')
            
            # Additional analysis
            print("\n📈 ADDITIONAL SCHEMA INSIGHTS:")
            print("─" * 40)
            
            # Parse the descriptions to count field types
            try:
                desc_data = json.loads(descriptions)
                print(f"✅ Generated descriptions for {len(desc_data)} columns")
                print(f"✅ DDL contains {ddl.count('`')} field definitions")
                print(f"✅ Schema ready for database implementation")
            except:
                print("✅ Schema generation completed (descriptions in text format)")
                
        except Exception as e:
            print(f"❌ Error in schema generation: {e}")
            import traceback
            print(f"Full error: {traceback.format_exc()}")
            
else:
    if not connection_ok:
        print("❌ Cannot test SAP schema generation - Claude API connection failed")
    elif 'sap' not in datasets:
        print(f"❌ SAP dataset not found. Available datasets: {list(datasets.keys()) if 'datasets' in globals() else 'None'}")
        print("Make sure the SAP CSV file exists at: /Users/tomasnagy/Scavenger AI/sap.csv")
    else:
        print("❌ Unknown error - cannot proceed with SAP testing")

print("\n🏁 SAP Dataset Schema Generation Test Complete")

🔍 COMPREHENSIVE SAP DATASET SCHEMA GENERATION TEST

🔗 Testing Claude API connection...
✅ Claude API connection successful

🎯 TESTING SAP DATASET SCHEMA GENERATION
──────────────────────────────────────────────────
📊 SAP Dataset Overview:
   📈 Rows: 100,000
   📋 Columns: 64
   💾 Memory: 25.5 MB
   🏷️ Column names: ['VBELN', 'GPAG', 'JPARVWGPAG', 'ANGDT', 'BNDDT', 'VBTYP', 'TRVOG', 'AUART', 'AUARTGRP', 'WAERK', 'NETWR', 'VKORG', 'VTWEG', 'SPART', 'VKGRP', 'VKBUR', 'EXEMPLART', 'GSBER', 'GSKST', 'KNUMV', 'FAKSP', 'KALSM', 'KURST', 'BSTNK', 'BSARK', 'BSTDK', 'BSTZD', 'IHREZ', 'STAFO', 'STWAE', 'XIMMATRIK', 'IMMATDAT', 'XSEPFKKOPF', 'XREMRECHT', 'REMSP', 'KORRGRD', 'REMVON', 'REMBIS', 'REFBELEG', 'REMDATUM', 'REMSCHEIN', 'IVWDATUM', 'POSNR_LAST', 'POSEX_LAST', 'KPOSN_LAST', 'XFKBASAUFT', 'XFKBASLIEF', 'XJKSOFAKT', 'ERFUSER', 'ERFDATE', 'ERFTIME', 'AENUSER', 'AENDATE', 'AENTIME', 'XWBZABO', 'XFKVDICHT', 'XSTATARC', 'XNOMESS', 'KALSM_AMO', 'XRENEWAL', 'AMORTN', 'REKLERGB', 'REKLTYP', 'REKLDAT

In [53]:
# IMPROVED SAP Dataset Schema Generation with Enhanced Error Handling
print("🚀 IMPROVED SAP DATASET SCHEMA GENERATION")
print("=" * 70)

# Enhanced function to handle large datasets
def analyze_large_dataset_schema(df: pl.DataFrame, table_name: str, max_columns_per_batch: int = 30) -> Tuple[str, str]:
    """Analyze schema for large datasets by processing in batches if needed"""
    
    total_columns = len(df.columns)
    print(f"📊 Dataset: {table_name} with {total_columns} columns")
    
    if total_columns <= max_columns_per_batch:
        # Process all columns at once
        return analyze_schema_with_claude(df, table_name, sample_size=3)
    else:
        # Process in batches
        print(f"⚠️ Large dataset detected ({total_columns} columns). Processing in batches...")
        
        all_schemas = {}
        batch_num = 1
        
        for i in range(0, total_columns, max_columns_per_batch):
            batch_columns = df.columns[i:i + max_columns_per_batch]
            batch_df = df.select(batch_columns)
            
            print(f"\n🔄 Processing batch {batch_num}: columns {i+1}-{min(i+max_columns_per_batch, total_columns)}")
            
            # Extract samples for this batch
            batch_samples = extract_column_samples(batch_df, sample_size=3)
            
            # Create prompt for this batch
            batch_prompt = prepare_claude_prompt(batch_samples, f"{table_name}_batch_{batch_num}")
            
            # Call Claude API
            print(f"🤖 Calling Claude API for batch {batch_num}...")
            batch_schema = call_claude_api(batch_prompt)
            
            if batch_schema and 'columns' in batch_schema:
                all_schemas.update(batch_schema['columns'])
                print(f"✅ Batch {batch_num} completed successfully")
            else:
                print(f"❌ Batch {batch_num} failed")
            
            batch_num += 1
        
        # Combine all schemas
        combined_schema = {'columns': all_schemas}
        ddl = generate_ddl_from_schema(combined_schema, table_name)
        descriptions = generate_column_descriptions(combined_schema)
        
        return ddl, descriptions

# Test with SAP dataset
if 'sap' in datasets:
    print("\n🎯 Testing improved SAP dataset schema generation...")
    
    # Test Claude connection first
    connection_ok = test_claude_connection()
    
    if connection_ok:
        try:
            sap_df = datasets['sap']
            print(f"\n📈 SAP Dataset: {sap_df.shape[0]:,} rows × {sap_df.shape[1]} columns")
            
            # Use the improved function
            ddl, descriptions = analyze_large_dataset_schema(sap_df, 'sap_sales_data', max_columns_per_batch=20)
            
            # Display results
            print("\n" + "=" * 80)
            print("🎉 IMPROVED SAP SCHEMA GENERATION COMPLETED!")
            print("=" * 80)
            
            display_schema_results(ddl, descriptions, 'sap_sales_data')
            
            # Validation
            try:
                desc_data = json.loads(descriptions)
                print(f"\n✅ Successfully generated schema for {len(desc_data)} columns")
                print(f"✅ DDL ready for database implementation")
                
                # Show sample of descriptions
                print(f"\n📝 Sample column descriptions:")
                for i, (col, desc) in enumerate(list(desc_data.items())[:5]):
                    print(f"   {i+1}. {col}: {desc[:100]}...")
                    
            except Exception as e:
                print(f"⚠️ Could not parse descriptions: {e}")
                
        except Exception as e:
            print(f"❌ Error in improved schema generation: {e}")
            import traceback
            traceback.print_exc()
    else:
        print("❌ Claude API connection failed")
else:
    print("❌ SAP dataset not found")

print("\n🏁 Improved SAP Schema Generation Test Complete")

🚀 IMPROVED SAP DATASET SCHEMA GENERATION

🎯 Testing improved SAP dataset schema generation...
✅ Claude API connection successful

📈 SAP Dataset: 100,000 rows × 64 columns
📊 Dataset: sap_sales_data with 64 columns
⚠️ Large dataset detected (64 columns). Processing in batches...

🔄 Processing batch 1: columns 1-20
🤖 Calling Claude API for batch 1...
✅ Batch 1 completed successfully

🔄 Processing batch 2: columns 21-40
🤖 Calling Claude API for batch 2...
✅ Batch 2 completed successfully

🔄 Processing batch 3: columns 41-60
🤖 Calling Claude API for batch 3...
✅ Batch 3 completed successfully

🔄 Processing batch 4: columns 61-64
🤖 Calling Claude API for batch 4...
✅ Batch 4 completed successfully

🎉 IMPROVED SAP SCHEMA GENERATION COMPLETED!

📋 CLAUDE-GENERATED SCHEMA FOR SAP_SALES_DATA

🗂️ DDL (Data Definition Language):
──────────────────────────────────────────────────
CREATE TABLE sap_sales_data (
  `VBELN` INT NOT NULL,
  `GPAG` DECIMAL(10,0) NOT NULL,
  `JPARVWGPAG` BOOLEAN NOT NULL,
 

In [ ]:
# COPY-PASTE FORMATTING FUNCTIONS
print("📋 COPY-PASTE FORMATTING FUNCTIONS")
print("=" * 60)

def format_for_copy_paste(ddl: str, descriptions: str, table_name: str, source_file: str = None) -> str:
    """Format results for easy copy-paste with timestamps and metadata"""
    
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    
    output = f"""-- =========================================
-- SCAVENGER AI - GENERATED SCHEMA
-- =========================================
-- Table Name: {table_name.upper()}
-- Generated: {timestamp}
-- Source File: {source_file if source_file else 'Unknown'}
-- Tool: Scavenger AI Schema Generator v1.0
-- =========================================

{ddl}

-- =========================================
-- COLUMN DESCRIPTIONS
-- =========================================
/*
Generated Column Descriptions (JSON Format):

{descriptions}
*/

-- =========================================
-- IMPLEMENTATION NOTES
-- =========================================
/*
1. Review data types for optimization
2. Add indexes as needed for performance
3. Consider adding primary key constraints
4. Validate NOT NULL constraints with business rules
5. Add foreign key relationships if applicable
*/
"""
    return output

def format_ddl_only(ddl: str, table_name: str) -> str:
    """Format only DDL for quick implementation"""
    
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    
    output = f"""-- Generated by Scavenger AI on {timestamp}
-- Table: {table_name.upper()}

{ddl}
"""
    return output

def format_descriptions_only(descriptions: str, table_name: str) -> str:
    """Format only descriptions for documentation"""
    
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    
    output = f"""-- COLUMN DESCRIPTIONS FOR {table_name.upper()}
-- Generated by Scavenger AI on {timestamp}

{descriptions}
"""
    return output

def generate_implementation_guide(ddl: str, descriptions: str, table_name: str) -> str:
    """Generate implementation guide with best practices"""
    
    try:
        desc_data = json.loads(descriptions)
        column_count = len(desc_data)
    except:
        column_count = ddl.count('`') // 2  # Estimate from backticks
    
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')
    
    guide = f"""-- IMPLEMENTATION GUIDE FOR {table_name.upper()}
-- Generated: {timestamp}
-- Columns: {column_count}

-- 1. CREATE TABLE STATEMENT
{ddl}

-- 2. RECOMMENDED INDEXES
-- Add these after table creation for better performance:
/*
CREATE INDEX idx_{table_name}_created ON {table_name} (created_at);
-- Add more indexes based on your query patterns
*/

-- 3. SAMPLE INSERT STATEMENT
/*
INSERT INTO {table_name} (
  -- List your columns here
) VALUES (
  -- Provide sample values
);
*/

-- 4. SAMPLE SELECT STATEMENTS
/*
-- Basic select
SELECT * FROM {table_name} LIMIT 10;

-- Count records
SELECT COUNT(*) FROM {table_name};

-- Check data quality
SELECT 
  COUNT(*) as total_rows,
  COUNT(*) - COUNT(column_name) as nulls_in_column
FROM {table_name};
*/

-- 5. COLUMN DESCRIPTIONS
{descriptions}
"""
    return guide

def save_formatted_output(ddl: str, descriptions: str, table_name: str, output_type: str = "complete"):
    """Save formatted output to file (simulated for notebook)"""
    
    if output_type == "complete":
        content = format_for_copy_paste(ddl, descriptions, table_name)
        filename = f"{table_name}_complete_schema.sql"
    elif output_type == "ddl":
        content = format_ddl_only(ddl, table_name)
        filename = f"{table_name}_ddl.sql"
    elif output_type == "descriptions":
        content = format_descriptions_only(descriptions, table_name)
        filename = f"{table_name}_descriptions.json"
    elif output_type == "guide":
        content = generate_implementation_guide(ddl, descriptions, table_name)
        filename = f"{table_name}_implementation_guide.sql"
    else:
        print(f"❌ Unknown output type: {output_type}")
        return
    
    print(f"📁 Generated {output_type} format for {table_name}")
    print(f"💾 Filename: {filename}")
    print(f"📊 Size: {len(content)} characters")
    
    # In a real implementation, you would save to file:
    # with open(filename, 'w') as f:
    #     f.write(content)
    
    return content, filename

def display_copy_paste_options(ddl: str, descriptions: str, table_name: str):
    """Display all copy-paste options with clear formatting"""
    
    print(f"\n📋 COPY-PASTE OPTIONS FOR {table_name.upper()}")
    print("=" * 70)
    
    # Option 1: Complete Package
    print("\n📦 1. COMPLETE PACKAGE (DDL + Descriptions + Metadata)")
    print("─" * 60)
    complete_output = format_for_copy_paste(ddl, descriptions, table_name)
    print(complete_output)
    
    # Option 2: DDL Only
    print("\n\n🔧 2. DDL ONLY (Quick Implementation)")
    print("─" * 60)
    ddl_output = format_ddl_only(ddl, table_name)
    print(ddl_output)
    
    # Option 3: Implementation Guide
    print("\n\n📖 3. IMPLEMENTATION GUIDE (Best Practices)")
    print("─" * 60)
    guide_output = generate_implementation_guide(ddl, descriptions, table_name)
    print(guide_output)
    
    print("\n" + "=" * 70)
    print("✅ All formats ready for copy-paste!")
    
    return {
        'complete': complete_output,
        'ddl_only': ddl_output,
        'implementation_guide': guide_output
    }

print("✅ Copy-paste formatting functions loaded successfully!")
print("Available functions:")
print("  - format_for_copy_paste()")
print("  - format_ddl_only()")
print("  - format_descriptions_only()")
print("  - generate_implementation_guide()")
print("  - display_copy_paste_options()")
print("  - save_formatted_output()")

In [ ]:
# OPTIMIZED SCHEMA GENERATION WITH AUTO-BATCHING (20 COLUMNS MAX)
print("🎯 OPTIMIZED AUTO-BATCHING SCHEMA GENERATION")
print("=" * 60)

def generate_schema_with_auto_batching(df: pl.DataFrame, table_name: str) -> Tuple[str, str]:
    """
    Generate schema with automatic batching for datasets.
    - Small datasets (≤20 columns): Single batch processing
    - Large datasets (>20 columns): Automatic 20-column batches
    """
    
    total_columns = len(df.columns)
    BATCH_SIZE = 20
    
    print(f"📊 Dataset '{table_name}': {df.shape[0]:,} rows × {total_columns} columns")
    
    if total_columns <= BATCH_SIZE:
        # Small dataset - process all at once
        print(f"✅ Small dataset detected. Processing all {total_columns} columns in single batch.")
        return analyze_schema_with_claude(df, table_name, sample_size=5)
    
    else:
        # Large dataset - process in batches of 20
        num_batches = (total_columns + BATCH_SIZE - 1) // BATCH_SIZE  # Ceiling division
        print(f"📦 Large dataset detected. Processing in {num_batches} batches of {BATCH_SIZE} columns each.")
        
        all_schemas = {}
        
        for batch_num in range(1, num_batches + 1):
            start_idx = (batch_num - 1) * BATCH_SIZE
            end_idx = min(start_idx + BATCH_SIZE, total_columns)
            batch_columns = df.columns[start_idx:end_idx]
            
            print(f"\n🔄 Batch {batch_num}/{num_batches}: Processing columns {start_idx + 1}-{end_idx}")
            print(f"   📝 Columns: {list(batch_columns)}")
            
            # Create batch DataFrame
            batch_df = df.select(batch_columns)
            
            # Extract samples for this batch
            batch_samples = extract_column_samples(batch_df, sample_size=5)
            
            # Create prompt for this batch
            batch_prompt = prepare_claude_prompt(batch_samples, f"{table_name}_batch_{batch_num}")
            
            # Call Claude API
            print(f"   🤖 Calling Claude API...")
            batch_schema = call_claude_api(batch_prompt)
            
            if batch_schema and 'columns' in batch_schema:
                all_schemas.update(batch_schema['columns'])
                print(f"   ✅ Batch {batch_num} completed successfully ({len(batch_schema['columns'])} columns processed)")
            else:
                print(f"   ❌ Batch {batch_num} failed - skipping")
        
        # Combine all schemas
        print(f"\n🔗 Combining results from all batches...")
        combined_schema = {'columns': all_schemas}
        ddl = generate_ddl_from_schema(combined_schema, table_name)
        descriptions = generate_column_descriptions(combined_schema)
        
        print(f"✅ Successfully processed {len(all_schemas)} columns across {num_batches} batches")
        return ddl, descriptions

# Test the optimized function with both datasets
print("\n🧪 TESTING AUTO-BATCHING FUNCTION")
print("-" * 50)

# Test Claude connection first
connection_ok = test_claude_connection()

if connection_ok and 'datasets' in globals() and datasets:
    
    # Test with Insurance dataset (small - should be single batch)
    if 'insurance' in datasets:
        print("\n📋 Testing with Insurance Dataset (Small):")
        print("=" * 45)
        try:
            insurance_ddl, insurance_desc = generate_schema_with_auto_batching(datasets['insurance'], 'insurance_data')
            display_schema_results(insurance_ddl, insurance_desc, 'insurance_data')
        except Exception as e:
            print(f"❌ Error with insurance dataset: {e}")
    
    # Test with SAP dataset (large - should be multi-batch)
    if 'sap' in datasets:
        print("\n📋 Testing with SAP Dataset (Large):")
        print("=" * 40)
        try:
            sap_ddl, sap_desc = generate_schema_with_auto_batching(datasets['sap'], 'sap_sales_data')
            display_schema_results(sap_ddl, sap_desc, 'sap_sales_data')
            
            # Show summary statistics
            try:
                desc_data = json.loads(sap_desc)
                print(f"\n📈 FINAL SUMMARY:")
                print(f"   ✅ Total columns processed: {len(desc_data)}")
                print(f"   ✅ DDL lines generated: {sap_ddl.count('`')}")
                print(f"   ✅ Schema ready for implementation")
                
                # Show first few column descriptions as examples
                print(f"\n📝 Sample Column Descriptions:")
                for i, (col, desc) in enumerate(list(desc_data.items())[:3]):
                    print(f"   {i+1}. {col}: {desc[:80]}...")
                    
            except Exception as e:
                print(f"⚠️ Could not parse final descriptions: {e}")
                
        except Exception as e:
            print(f"❌ Error with SAP dataset: {e}")
            import traceback
            traceback.print_exc()
else:
    if not connection_ok:
        print("❌ Claude API connection failed")
    else:
        print("❌ No datasets available for testing")

print("\n🏁 Auto-Batching Schema Generation Test Complete")

## Component Three: Auto-Batching Schema Generation

### Goal
Intelligently generate SQL schemas and column descriptions using Claude AI with automatic batch processing for optimal performance.

### Key Features

- **Smart Batching Logic:**
  - Small datasets (≤20 columns): Single batch processing
  - Large datasets (>20 columns): Automatic 20-column batches
  
- **Comprehensive Output:**
  - Production-ready DDL (Data Definition Language)
  - Professional column descriptions in JSON format
  - Progress tracking and error handling

### Technology Stack

- **Claude AI API:** Advanced schema analysis and type inference
- **Polars:** High-performance data sampling
- **Batch Processing:** Optimized for large datasets like SAP (64 columns)

### Success Criteria

- ✅ Accurate SQL type mapping (INT, VARCHAR, DECIMAL, etc.)
- ✅ Professional business-oriented column descriptions
- ✅ Reliable processing of both small and large datasets
- ✅ Database-ready DDL generation